#  Consignes

## Description

récupérer un jeu de données qui vous intéresse (exemple de source : www.data.gouv.fr) : Ecrire le code qui le pousse  proprement dans une base de données mongo, et puis écrire les queries qui permettent d’extraire les données les plus intéressantes (des commentaires sont appréciés)

Base de donnée : Top 500 des albums des Rolling Stones
Source : https://data.world/notgibs/rolling-stones-top-500-albums/workspace/file?filename=albumlist.csv


In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo:27017")
database = client['exercices']
collection = database['albums']

In [70]:
df_al = pd.read_csv("./data/albumlist.csv", encoding = "ISO-8859-1", engine='python')

### Netoyer les données

In [71]:
l_d =df_al.to_dict(orient='records')

### Importer les données

In [82]:
collection.delete_many({})
collection.insert_many(l_d)

## Requête 1  
    
Récupérer les albums réalisés par "Metallica"

In [52]:
list(collection.find({'Artist': 'Metallica'}))

[{'_id': ObjectId('602f9a85b4cf2c001163d17e'),
  'Number': 167,
  'Year': 1986,
  'Album': 'Master of Puppets',
  'Artist': 'Metallica',
  'Genre': 'Rock',
  'Subgenre': 'Thrash, Speed Metal'},
 {'_id': ObjectId('602f9a85b4cf2c001163d1d6'),
  'Number': 255,
  'Year': 1991,
  'Album': 'Metallica ("The Black Album")',
  'Artist': 'Metallica',
  'Genre': 'Rock',
  'Subgenre': 'Heavy Metal'}]

## Requête 2

2) Le nombre d'album du style "Soul"

In [54]:
collection.create_index([("Genre", "text")])

next(collection.aggregate([
        {"$match":{"$text":{"$search" : "Soul"}}},
        {"$count" : "nb_album"}
    ]))

{'nb_album': 80}

## Requête 3

Compter le nombre d'albums pour chaque artiste.

In [40]:
list(collection.aggregate([
    {"$group" : {
        "_id" : "$Artist", 
        "nb_album" : {"$sum" : 1}
    }}
]))

[{'_id': 'The Modern Lovers', 'nb_album': 1},
 {'_id': 'Sam Cooke', 'nb_album': 2},
 {'_id': 'Ramones', 'nb_album': 2},
 {'_id': 'Lou Reed', 'nb_album': 2},
 {'_id': 'Steve Earle', 'nb_album': 1},
 {'_id': 'MGMT', 'nb_album': 1},
 {'_id': 'Madonna', 'nb_album': 3},
 {'_id': 'Fleetwood Mac', 'nb_album': 2},
 {'_id': 'The Stone Roses', 'nb_album': 1},
 {'_id': 'Pixies', 'nb_album': 2},
 {'_id': 'Creedence Clearwater Revival', 'nb_album': 2},
 {'_id': 'Jay Z', 'nb_album': 3},
 {'_id': 'Toots & The Maytals', 'nb_album': 1},
 {'_id': 'Quicksilver Messenger Service', 'nb_album': 1},
 {'_id': 'Metallica', 'nb_album': 2},
 {'_id': 'The Kinks', 'nb_album': 3},
 {'_id': 'Linda Ronstadt', 'nb_album': 1},
 {'_id': 'Professor Longhair', 'nb_album': 1},
 {'_id': 'Black Flag', 'nb_album': 1},
 {'_id': 'Dusty Springfield', 'nb_album': 1},
 {'_id': 'Funkadelic', 'nb_album': 2},
 {'_id': 'Queen', 'nb_album': 1},
 {'_id': 'De La Soul', 'nb_album': 1},
 {'_id': 'Lil Wayne', 'nb_album': 1},
 {'_id': 'Rod S

## Requête 4

Les albums Blues des années 70

In [73]:
collection.create_index([("Genre", "text")])

list(collection.find({"$text":{"$search" : "Blues"},"Year":{"$gte":1970, "$lt": 1980}}))

[{'_id': ObjectId('602ff843b4cf2c001163d4c2'),
  'Number': 499,
  'Year': 1971,
  'Album': 'Live in Cook County Jail',
  'Artist': 'B.B. King',
  'Genre': 'Blues',
  'Subgenre': 'Electric Blues'},
 {'_id': ObjectId('602ff843b4cf2c001163d41d'),
  'Number': 334,
  'Year': 1979,
  'Album': 'Squeezing Out Sparks',
  'Artist': 'Graham Parker & The Rumour',
  'Genre': 'Rock, Blues',
  'Subgenre': 'None'},
 {'_id': ObjectId('602ff843b4cf2c001163d34c'),
  'Number': 125,
  'Year': 1971,
  'Album': 'Pearl',
  'Artist': 'Janis Joplin',
  'Genre': 'Rock, Blues',
  'Subgenre': 'Blues Rock'},
 {'_id': ObjectId('602ff843b4cf2c001163d300'),
  'Number': 49,
  'Year': 1971,
  'Album': 'At Fillmore East',
  'Artist': 'The Allman Brothers Band',
  'Genre': 'Rock, Blues',
  'Subgenre': 'Blues Rock'},
 {'_id': ObjectId('602ff843b4cf2c001163d3ad'),
  'Number': 222,
  'Year': 1972,
  'Album': 'New Orleans Piano',
  'Artist': 'Professor Longhair',
  'Genre': 'Funk / Soul, Blues',
  'Subgenre': 'Piano Blues'}]

## Requête 5

Les artistes ayant fait le plus d'albums

In [51]:
list(collection.aggregate([
    {"$group":{
        "_id" : "$Artist", 
        "nb_album" : {"$sum" :1}}
    },
    {"$sort" : {"nb_album" : -1}},
    {"$limit" : 5}
]))

[{'_id': 'The Beatles', 'nb_album': 10},
 {'_id': 'The Rolling Stones', 'nb_album': 10},
 {'_id': 'Bob Dylan', 'nb_album': 10},
 {'_id': 'Bruce Springsteen', 'nb_album': 8},
 {'_id': 'The Who', 'nb_album': 7}]

## Requête 6 

L'année ayant le plus d'album sortie

In [60]:
list(collection.aggregate([
    {"$group":{
        "_id" : "$Year", 
        "nb_album" : {"$sum" :1}}
    },
    {"$sort" : {"nb_album" : -1}},
    {"$limit" : 1}
]))

[{'_id': 1970, 'nb_album': 26}]

## Requête 7

Ajouter l'album Hybrid Theory & Meteora de Linkin Park

In [83]:
list(collection.find({"Artist": "Linkin Park"}))

[]

In [84]:
collection.insert_many([
    {
        'Number': 501,
        'Year': 2000,
        'Album': 'Hybrid Theory',
        'Artist': 'Linkin Park',
        'Genre': 'Rock, Metal, Rap Metal',
        'Subgenre': 'Nu Metal, Metal Alterative, Rap Rock'
    },
    {
        'Number': 502,
        'Year': 2003,
        'Album': 'Meteora',
        'Artist': 'Linkin Park',
        'Genre': 'Rock, Metal, Rap Metal, Metal Alternative',
        'Subgenre': 'Nu Metal, Metal Alterative, Rap Rock'
    }
])


In [85]:
list(collection.find({"Artist": "Linkin Park"}))

[{'_id': ObjectId('602ffaccb4cf2c001163d4c9'),
  'Number': 501,
  'Year': 2000,
  'Album': 'Hybrid Theory',
  'Artist': 'Linkin Park',
  'Genre': 'Rock, Metal, Rap Metal',
  'Subgenre': 'Nu Metal, Metal Alterative, Rap Rock'},
 {'_id': ObjectId('602ffaccb4cf2c001163d4ca'),
  'Number': 502,
  'Year': 2003,
  'Album': 'Meteora',
  'Artist': 'Linkin Park',
  'Genre': 'Rock, Metal, Rap Metal, Metal Alternative',
  'Subgenre': 'Nu Metal, Metal Alterative, Rap Rock'}]

## Requête 8

Ajouter l'album The End Is Where We Begin & Welcome to the Masquerade de TFK

In [86]:
list(collection.find({"Artist": "TFK"}))

[]

In [87]:
collection.insert_many([
    {
        'Number': 503,
        'Year': 2009,
        'Album': 'Welcome to the Masquerade',
        'Artist': 'TFK',
        'Genre': 'Rock, Metal, Metal Alternative',
        'Subgenre': 'Christian Rock, Metal Alterative, Rap Rock'
    },
    {
        'Number': 504,
        'Year': 2013,
        'Album': 'The End Is Where We Begin',
        'Artist': 'TFK',
        'Genre': 'Rock, Metal, Rap Metal, Metal Alternative',
        'Subgenre': 'Christian Rock, Metal Alterative'
    }
])

In [88]:
list(collection.find({"Artist": "TFK"}))

[{'_id': ObjectId('602ffad7b4cf2c001163d4cb'),
  'Number': 503,
  'Year': 2009,
  'Album': 'Welcome to the Masquerade',
  'Artist': 'TFK',
  'Genre': 'Rock, Metal, Metal Alternative',
  'Subgenre': 'Christian Rock, Metal Alterative, Rap Rock'},
 {'_id': ObjectId('602ffad7b4cf2c001163d4cc'),
  'Number': 504,
  'Year': 2013,
  'Album': 'The End Is Where We Begin',
  'Artist': 'TFK',
  'Genre': 'Rock, Metal, Rap Metal, Metal Alternative',
  'Subgenre': 'Christian Rock, Metal Alterative'}]

## Requête 9

Renomer l'artiste TFK en Thousand Foot Krutch

In [93]:
collection.update_many({"Artist":"TFK"}, {"$set":{"Artist":"Thousand Foot Krutch"}})

In [96]:
list(collection.find({"Artist": "TFK"}))

[]

In [94]:
list(collection.find({"Artist": "Thousand Foot Krutch"}))

[{'_id': ObjectId('602ffad7b4cf2c001163d4cb'),
  'Number': 503,
  'Year': 2009,
  'Album': 'Welcome to the Masquerade',
  'Artist': 'Thousand Foot Krutch',
  'Genre': 'Rock, Metal, Metal Alternative',
  'Subgenre': 'Christian Rock, Metal Alterative, Rap Rock'},
 {'_id': ObjectId('602ffad7b4cf2c001163d4cc'),
  'Number': 504,
  'Year': 2013,
  'Album': 'The End Is Where We Begin',
  'Artist': 'Thousand Foot Krutch',
  'Genre': 'Rock, Metal, Rap Metal, Metal Alternative',
  'Subgenre': 'Christian Rock, Metal Alterative'}]

## Requête 10

Supprimer l'album Welcome to the Masquerade de Thousand Foot Krutch

In [98]:
list(collection.find({"Album": "Welcome to the Masquerade"}))

[{'_id': ObjectId('602ffad7b4cf2c001163d4cb'),
  'Number': 503,
  'Year': 2009,
  'Album': 'Welcome to the Masquerade',
  'Artist': 'Thousand Foot Krutch',
  'Genre': 'Rock, Metal, Metal Alternative',
  'Subgenre': 'Christian Rock, Metal Alterative, Rap Rock'}]

In [99]:
collection.delete_one({"Album": "Welcome to the Masquerade"})

In [100]:
list(collection.find({"Album": "Welcome to the Masquerade"}))

[]